## SQL Alchemy Class definition

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

article = Article(title, article_text, url, most_common)
session.add(article)

session.commit()

In [ ]:
for instance in session.query(Article).order_by(Article.id):
    print(instance.title, instance.url)

## Scraper code

In [1]:
from libs.article import Article

import re
import requests

from bs4 import BeautifulSoup
from sqlalchemy.orm import sessionmaker

from libs.sqlcreator import create_alchemy_engine
from libs.multi_thread import multi_thread

In [2]:
import string

from collections import Counter
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

def get_most_common(text):
    exclude = set(string.punctuation)
    text_nopunct = ''.join(ch for ch in text if ch not in exclude)

    words = text_nopunct.lower().split(" ")
    words = [word for word in words if word not in ENGLISH_STOP_WORDS and len(word) > 1]

    return [word for word,count in list(Counter(words).most_common(15))]


with open("npr_article_572945894.txt") as file_hdl:
    article_text = file_hdl.read()
    
title = article_text.split("\n")[0]
url = "npr.org/testing"

most_common = get_most_common(article_text)


In [3]:
def get_text(soup):
    text = ""
    for paragraph in soup.find_all('p'):
        if not paragraph.has_attr('class') and not paragraph.findChildren('b'):
            text += paragraph.get_text() + "\n\n"
            
    return text

In [4]:
def get_npr_urls(soup):
    
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    urls = []
    for link in soup.find_all('a', href=True):
        url = link['href']
        if (url.startswith('https://www.npr.org/')):
            if re.search(pattern, url):
                urls.append(url)
        
    return urls

In [5]:
def get_title(soup):
    title = ""
    #find href with class == title, then get b
    for link in soup.find_all('a'):
        if link.has_attr('class'):
            if link['class'][0] == "title":
                title = link.findChildren('b')[0].get_text()
    
    return title

In [6]:
def get_details(url):
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    matchObject = re.search(pattern, url, flags=0)
    match_split = matchObject[0].split("/")
    date = match_split[0] + "-" + match_split[1] + "-" + match_split[2]
    article_id = match_split[3]
    
    return date, article_id

In [7]:
def scrape_url(url):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    links = get_npr_urls(soup)

    urls = [x for x in get_npr_urls(soup)]

    text = get_text(soup)

    return_dict = {"urls": links}
    if len(text) > 100:
        title = get_title(soup)
        date, article_id = get_details(url)

        return_dict["article"] = Article(title, text, url, get_most_common(text), date)
        
    return return_dict

In [11]:
def is_url_in_db(session, url):
    return session.query(Article).filter(Article.url == url).count() > 0

is_url_in_db(session, "https://www.npr.org")

False

In [8]:
article_dict = scrape_url("https://www.npr.org/2018/05/04/608323118/in-wake-of-school-shooting-trump-pence-to-address-nra")
article_dict["article"]

In [2]:
##############
## 

start_url = "https://www.npr.org/"
traversed_urls = set()

to_traverse = set()
to_traverse.add(start_url)

engine = create_alchemy_engine()


Session = sessionmaker(bind=engine)
session = Session()

In [12]:
count = 0
while len(to_traverse) > 0 and count < 10000:

    ## Create a list of URLS to traverse
    curr_traverse = []
    while len(curr_traverse) < 20 and len(to_traverse) > 0:
        url = to_traverse.pop()
        if is_url_in_db(session, url):
            continue
        curr_traverse.append(url)
    
    ## Pass that list to multi-threading
    results = multi_thread(scrape_url, curr_traverse, 10)
    
    ## Multi-threading should return dictionaries mapping to results and to discovered URLs
    
    
    ## Create class for DB that maintains unique set of to_traverse and traverse
    for result in results:
        result_dict = result[1]
        if "article" in result_dict:
            session.add(result_dict["article"])
            count += 1
        
        to_traverse = to_traverse.union(set(result_dict["urls"]) - traversed_urls)
        
    session.commit()        
    count += 1
              

KeyboardInterrupt: 

In [3]:
from sqlalchemy import desc

#for instance in session.query(Article).order_by(desc(Article.date)):
#    print(instance.title, instance.url)

for instance in session.query(Article).order_by(desc(Article.date)).limit(10).offset(10):
    print(instance.title, instance.url)

Gun Control Activists Protest NRA In Dallas https://www.npr.org/2018/05/05/608723620/gun-control-activists-protest-nra-in-dallas
The Trump And Rudy Show https://www.npr.org/2018/05/05/608723613/the-trump-and-rudy-show
How To Make Movies 'Like Brothers' https://www.npr.org/2018/05/05/608723662/how-to-make-movies-like-brothers
Zora Neale Hurston's 'Barracoon' Gets Published, More Than 60 Years Later https://www.npr.org/2018/05/05/608723606/zora-neale-hurstons-barracoon-gets-published-more-than-60-years-later
College Apologizes After Native American Students' Visit Is Sidelined By Police https://www.npr.org/sections/thetwo-way/2018/05/04/608533284/college-apologizes-after-native-american-students-visit-is-sidelined-by-police
Three More Women Come Forward To Accuse R. Kelly Of Abuse https://www.npr.org/sections/therecord/2018/05/04/608608800/three-more-women-come-forward-to-accuse-r-kelly-of-abuse
GOP Primaries Focus On Candidates' Loyalty To President Trump https://www.npr.org/2018/05/04/

In [5]:
article = [x for x in session.query(Article).filter(Article.id == 1148)][0]
article

In [19]:
type(article)

sqlalchemy.orm.query.Query

In [6]:
session.query(Article).all().count()

TypeError: count() takes exactly one argument (0 given)